Q1. Install MLflow

In [6]:

! mlflow --version


mlflow, version 2.13.1


In [7]:
## Exploratory Data Analysis (EDA)
print(f"1, Data Dimension: {jan_df.shape[0]} rows | {jan_df.shape[1]} columns \n")

print(f"2, Duration Standard Deviation: {jan_df['duration'].std()} \n")

print(f"3, Outlier Proportion: {clean_prop} \n")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1393k  100 1393k    0     0  33.1M      0 --:--:-- --:--:-- --:--:-- 33.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1497k  100 1497k    0     0  3707k      0 --:--:-- --:--:-- --:--:-- 3707k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1690k  100 1690k    0     0  4507k      0 --:--:-- --:--:-- --:--:-- 4495k


In [8]:
## Apply One-Hot Encoding
ml_df = filtered_duration[['PULocationID', 'DOLocationID']].astype(str)
ml_df['duration'] = filtered_duration['duration']
ml_df.head()

,PULocationID,DOLocationID,duration
0,161,141,8.433333
1,43,237,6.316667
2,48,238,12.750000
3,138,7,9.616667
4,107,79,10.833333


In [9]:
## Dictionaries
dicts_train = ml_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dicts_train[1:5]

[{'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [10]:
vec = DictVectorizer(sparse = True)
feature_matrix = vec.fit_transform(dicts_train)

print(f"4, Dimension of feature_matrix: {feature_matrix.shape} \n")

4, Dimension of feature_matrix: (3009173, 515) 



In [11]:
## Linear Regression Model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y = ml_df['duration']

model = LinearRegression()
model.fit(feature_matrix, y)
y_pred = model.predict(feature_matrix)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"5, RMSE: {rmse}")

5, RMSE: 7.649261027919939


In [13]:
val_df = pd.read_parquet("./data/feb_yellow.parquet")
val_df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]] = val_df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].apply(pd.to_datetime)
val_df["duration"] = (val_df["tpep_dropoff_datetime"] - val_df["tpep_pickup_datetime"]).dt.total_seconds()/60
val_df = val_df[val_df['duration'].between(1,60)]

val_df.shape

(2855951, 20)